In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)

    for dim in ['band', 'g']: 
        if dim in ds: ds = ds.sum(dim)

    for dim in ['igg', 'g', 'i']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim) 
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10)),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    for dim in ['band', 'g']:
        if dim in ds: ds = ds.sum(dim)
        
    for dim in ['i', 'igg', 'g']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=10), ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

if 'ng_refs' in PARAM_LBLNEW: 
    doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))
else:
    doc = client.lblnew.overlap_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_flux']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolr']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wflux']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolr']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[6],4
commitnumber,1013f91,5014a19
dv,-,0.001
molecule,"{'co2': 0.0004, 'h2o': 'atmpro'}","{'h2o': 'atmpro', 'co2': 0.0004}"
nv,-,1000
tsfc,294,294


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.000755                    1 -0.000767
0.000750        2 -0.000757                    2 -0.000788
0.001052        3 -0.000761                    3 -0.000768
0.001476        4 -0.000765                    4 -0.000770
0.002070        5 -0.000768                    5 -0.000776
0.002904        6 -0.000771                    6 -0.000780
0.004074        7 -0.000773                    7 -0.000785
0.005714        8 -0.000774                    8 -0.000787
0.008015        9 -0.000775                    9 -0.000791
0.011243       10 -0.000775                   10 -0.000795
0.015771       11 -0.000775                   11 -0.000802
0.022122       12 -0.000774                   12 -0.000808
0.031031       13 -0.000772                   13 -0.000811
0.043528       14 -0.000769                   14 -0.000815
0.061057       15 -0.000754                   15 -0.000821
0.085645       16 -0.000662                   16 -0.000797
0.120136       17 -0.000348                   17 -0.000646
0.168516       18  0.000314                   18 -0.000266
0.236378       19  0.001511                   19  0.000460
0.331549       20  0.003506                   20  0.001697
0.465100       21  0.006652                   21  0.003695
0.652400       22  0.011390                   22  0.006995
0.915100       23  0.016555                   23  0.010684
1.283650       24  0.016679                   24  0.010906
1.800600       25  0.011979                   25  0.007891
2.525700       26  0.007478                   26  0.004930
3.542800       27  0.004200                   27  0.002708
4.969550       28  0.002005                   28  0.001184
6.970850       29  0.000636                   29  0.000199
9.778100       30 -0.000197                   30 -0.000425
13.715850      31 -0.000709                   31 -0.000825
19.239350      32 -0.000999                   32 -0.001059
26.987250      33 -0.001132                   33 -0.001172
37.855300      34 -0.001193                   34 -0.001231
53.100050      35 -0.001230                   35 -0.001265
73.887500      36 -0.001254                   36 -0.001286
97.662500      37 -0.001283                   37 -0.001311
121.437500     38 -0.001322                   38 -0.001329
145.212500     39 -0.001359                   39 -0.001313
168.987500     40 -0.001407                   40 -0.001335
192.762500     41 -0.001500                   41 -0.001392
216.537500     42 -0.001611                   42 -0.001544
240.312500     43 -0.001667                   43 -0.001659
264.087500     44 -0.001620                   44 -0.001707
287.862500     45 -0.001396                   45 -0.001650
311.637500     46 -0.000936                   46 -0.001395
335.412500     47 -0.000216                   47 -0.000900
359.187500     48  0.000831                   48 -0.000146
382.962500     49  0.002270                   49  0.000952
406.737500     50  0.004162                   50  0.002472
430.512500     51  0.006554                   51  0.004465
454.287500     52  0.009323                   52  0.006906
478.062500     53  0.012514                   53  0.009861
501.837500     54  0.016222                   54  0.013423
525.612500     55  0.020489                   55  0.017479
549.387500     56  0.025355                   56  0.022217
573.162500     57  0.030879                   57  0.027693
596.937500     58  0.038498                   58  0.035229
620.712500     59  0.047196                   59  0.043959
644.487500     60  0.056634                   60  0.053324
668.262500     61  0.066782                   61  0.063221
692.037500     62  0.077563                   62  0.073666
715.812500     63  0.088988                   63  0.084750
739.587500     64  0.101070                   64  0.096584
763.362500     65  0.113787                   65  0.109338
78

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -60.098092  0.000000e+00 -60.098092 -60.167705   
0.000624    2     -60.098092  3.228642e-08 -60.098092 -60.167705   
0.000876    3     -60.098092  4.552040e-08 -60.098092 -60.167705   
0.001229    4     -60.098092  6.477995e-08 -60.098092 -60.167705   
0.001723    5     -60.098092  9.287635e-08 -60.098092 -60.167705   
0.002417    6     -60.098092  1.338518e-07 -60.098092 -60.167705   
0.003391    7     -60.098092  1.936090e-07 -60.098092 -60.167705   
0.004757    8     -60.098093  2.807487e-07 -60.098092 -60.167705   
0.006672    9     -60.098093  4.077944e-07 -60.098092 -60.167706   
0.009359    10    -60.098093  5.930048e-07 -60.098093 -60.167706   
0.013128    11    -60.098094  8.629729e-07 -60.098093 -60.167707   
0.018415    12    -60.098095  1.256442e-06 -60.098094 -60.167707   
0.025830    13    -60.098096  1.829843e-06 -60.098094 -60.167709   
0.036232    14    -60.098098  2.665381e-06 -60.098095 -60.167710   
0.050823    15    -60.098100  3.885774e-06 -60.098096 -60.167713   
0.071291    16    -60.098104  5.812379e-06 -60.098098 -60.167716   
0.100000    17    -60.098110  9.499050e-06 -60.098101 -60.167721   
0.140271    18    -60.098120  1.741495e-05 -60.098102 -60.167730   
0.196760    19    -60.098134  3.430469e-05 -60.098100 -60.167745   
0.275997    20    -60.098156  6.986083e-05 -60.098086 -60.167766   
0.387100    21    -60.098183  1.435771e-04 -60.098040 -60.167795   
0.543100    22    -60.098211  2.941798e-04 -60.097917 -60.167833   
0.761700    23    -60.098218  5.956169e-04 -60.097622 -60.167872   
1.068500    24    -60.098166  1.145355e-03 -60.097020 -60.167895   
1.498800    25    -60.098078  1.908093e-03 -60.096170 -60.167923   
2.102400    26    -60.098044  2.730349e-03 -60.095314 -60.168017   
2.949000    27    -60.098113  3.549114e-03 -60.094564 -60.168214   
4.136600    28    -60.098319  4.346064e-03 -60.093973 -60.168543   
5.802500    29    -60.098697  5.119628e-03 -60.093577 -60.169033   
8.139200    30    -60.099286  5.885211e-03 -60.093401 -60.169724   
11.417000   31    -60.100140  6.662741e-03 -60.093478 -60.170663   
16.014700   32    -60.101326  7.461691e-03 -60.093864 -60.171914   
22.464000   33    -60.102922  8.294720e-03 -60.094627 -60.173552   
31.510500   34    -60.105062  9.221779e-03 -60.095840 -60.175710   
44.200100   35    -60.107967  1.033319e-02 -60.097633 -60.178615   
62.000000   36    -60.111913  1.168611e-02 -60.100227 -60.182550   
85.775000   37    -60.117034  1.327492e-02 -60.103759 -60.187663   
109.550000  38    -60.122163  1.479036e-02 -60.107372 -60.192782   
133.325000  39    -60.127427  1.633215e-02 -60.111095 -60.197965   
157.100000  40    -60.132819  1.789733e-02 -60.114922 -60.203084   
180.875000  41    -60.138445  1.955927e-02 -60.118886 -60.208339   
204.650000  42    -60.144739  2.162874e-02 -60.123111 -60.214063   
228.425000  43    -60.152245  2.459667e-02 -60.127648 -60.221003   
252.200000  44    -60.161276  2.893444e-02 -60.132342 -60.229421   
275.975000  45    -60.172229  3.532478e-02 -60.136904 -60.239632   
299.750000  46    -60.185674  4.483962e-02 -60.140835 -60.252145   
323.525000  47    -60.201506  5.803656e-02 -60.143470 -60.266919   
347.300000  48    -60.219720  7.564197e-02 -60.144078 -60.283781   
371.075000  49    -60.240526  9.878799e-02 -60.141738 -60.303028   
394.850000  50    -60.264136  1.287916e-01 -60.135344 -60.324987   
418.625000  51    -60.290746  1.671232e-01 -60.123623 -60.349807   
442.400000  52    -60.320551  2.153858e-01 -60.105165 -60.377545   
466.175000  53    -60.353353  2.744454e-01 -60.078908 -60.408149   
489.950000  54    -60.389078  3.454133e-01 -60.043664 -60.441658   
513.725000  55    -60.427800  4.298229e-01 -59.997977 -60.478232   
537.500000  56    -60.469577  5.293027e-01 -59.940274 -60.517591   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')